In [ ]:
import numpy as np
import xarray as xr
import pickle
import pandas as pd
import time
import os

from collections import OrderedDict

import datetime
from sys import getsizeof,path

import matplotlib.pyplot as plt
import matplotlib

path.append("../src")
from Splines import CentripetalCatmullRomSpline_splitControls,Spline
import Cases
from Population import ImportPopulation
from ModelParams import ObservedData,ModelParam



In [ ]:
matplotlib.rcParams['figure.figsize'] = [12, 6]

In [ ]:
df = pd.read_csv("../Data/Deaths/COVID-19_Todesfaelle_10y_210930.csv",sep=";",na_values="<4")
df = df.fillna(2).astype("int64")
df.index = range(10,len(df)+10)

f = {"AG 90+ Jahre":90}
for i in range(9):
    f["AG %d-%d Jahre"%(i*10,i*10+9)] = i*10

week_of_death = df.rename(columns=f)
print(week_of_death)

In [ ]:
week_of_death.filter([30,40,50,60,70,80,90]).plot()
plt.xticks([1,14,27,40,54,66,79,92,53*2],labels=[2020,"Q2","Q3","Q4",2021,"Q2","Q3","Q4",2022])
_ = plt.title("deaths by week of first case-report")

In [ ]:
df = pd.read_csv("../Data/Cases/SitRep_RKI/Hospitalisierung_Altersverteilung_210930.csv",sep=";",header=4)
#df = df.astype("int64")

f = {"Fälle A00..04":0,"Fälle A05..14":5,"Fälle A15..34":15,"Fälle A35..59":35,"Fälle A60..79":60,"Fälle A80+":80}
df = df.rename(columns=f)
df.index = range(10,len(df)+10)
meldedatum_hospitalization = df.filter(["Meldejahr","Meldewoche"]+list(f.values())).fillna(0).astype("int64")

meldedatum_hospitalization.filter([0,5,15,35,60,80]).plot()
plt.xticks([1,14,27,40,54,66,79,92,53*2],labels=[2020,"Q2","Q3","Q4",2021,"Q2","Q3","Q4",2022])
plt.title("Hospitalizations by week of first case-report (Meldedatum)")

print(meldedatum_hospitalization)

In [ ]:
plt.plot(meldedatum_hospitalization.filter([60]),"--",color="tab:red",label="hospitalizations 60-79")
plt.plot(meldedatum_hospitalization.filter([80]),color="tab:red",label="hospitalizations 80+")

plt.plot(week_of_death[60]+week_of_death[70],"--",color="tab:blue",label="deaths 60-79")
plt.plot(week_of_death[80]+week_of_death[90],color="tab:blue",label="deaths 80+")
#plt.plot(week_of_death.filter([80]),color="tab:blue")
plt.title("Comparison of Hospitalizations and deaths by week of first case-report (Meldedatum)")
plt.legend()
_ = plt.xticks([1,14,27,40,54,66,79,92,53*2],labels=[2020,"Q2","Q3","Q4",2021,"Q2","Q3","Q4",2022])

In [ ]:
path.append("../Data/Deaths/SitRep_RKI")
from CopiedData import DeathsPerAG

In [ ]:
# cuml. reportet death at each tuesday
DeathsPerAG()

In [ ]:
df = pd.read_csv("../Data/Deaths/DeathsRKI_fromArcgis.csv",sep=",",parse_dates=["Meldedatum","Refdatum","TodesMeldedatum"])
df = df.filter(["Altersgruppe","Geschlecht","AnzahlFall","AnzahlTodesfall","TodesMeldedatum"])

for k,v in zip(("A00-A04","A05-A14","A15-34","A35-A59","A60-A79","A80+"),(0,5,15,35,60,80)):
    ages[k] = v
data = np.zeros((n,2,len(ages),),dtype="int64")

weeks = pd.date_range("2019-12-30", periods=n, freq="W-MON")
for i,w1,w2 in zip(range(0,n),weeks[:-1],weeks[1:]):
    wmask = (df["TodesMeldedatum"] >= w1) & (df["TodesMeldedatum"] < w2)
    reported_deaths_in_week = df[wmask]
    
    for j,sex in enumerate(["M","W"]):
        reported_deaths_in_week_sex = reported_deaths_in_week[reported_deaths_in_week["Geschlecht"]==sex]
        for k,age in enumerate(ages.keys()):
            reported_deaths_in_week_sex_age = reported_deaths_in_week_sex[reported_deaths_in_week_sex["Altersgruppe"]==age]
            deaths = reported_deaths_in_week_sex_age["AnzahlTodesfall"].sum()
            data[i,j,k] = deaths

reported_deaths = xr.DataArray(data,dims=["week","sex","age"],coords={"week":range(1,n+1),"sex":["M","W"],"age":list(ages.values())})

In [ ]:
reported_deaths.sum("sex").sel(week=range(10,92))

In [ ]:
df = pd.read_csv("../Data/Cases/RKI_COVID19_211005.csv",sep=",",parse_dates=["Meldedatum","Refdatum"])
df = df.filter(["Altersgruppe","Geschlecht","AnzahlFall","AnzahlTodesfall","Meldedatum","Refdatum","IstErkrankungsbeginn"])

In [ ]:
n = 110
ages = OrderedDict()
for k,v in zip(("A00-A04","A05-A14","A15-34","A35-A59","A60-A79","A80+"),(0,5,15,35,60,80)):
    ages[k] = v
data = np.zeros((n,2,len(ages),2,),dtype="int64")

weeks = pd.date_range("2019-12-30", periods=n, freq="W-MON")
for i,w1,w2 in zip(range(0,n),weeks[:-1],weeks[1:]):
    wmask = (df["Meldedatum"] >= w1) & (df["Meldedatum"] < w2)
    if True:#i == 20:
        cases_in_week = df[wmask]
        for j,sex in enumerate(["M","W"]):
            cases_in_week_sex = cases_in_week[cases_in_week["Geschlecht"]==sex]
            for k,age in enumerate(ages.keys()):
                cases_in_week_sex_age = cases_in_week_sex[cases_in_week_sex["Altersgruppe"]==age]
                cases,deaths = cases_in_week_sex_age["AnzahlFall"].sum(),cases_in_week_sex_age["AnzahlTodesfall"].sum()
                data[i,j,k] = [cases,deaths]

cases = xr.DataArray(data,dims=["week","sex","age","status"],coords={"week":range(1,n+1),"sex":["M","W"],"age":list(ages.values()),"status":["cases","deaths"]})


In [ ]:
meldedatum_hospitalization[60].sum()
cases.sum("sex").sel(age=60,status="cases",week=range(10,92))

In [ ]:
meldedatum_hospitalization[60]

In [ ]:
df = pd.read_csv("../Data/ICU/DIVI/bund-altersstruktur-zeitreihe_210930.csv",sep=",",parse_dates=["Datum"],index_col=["Datum"])
df.index = df.index.tz_convert(None)
df = df.resample("W-MON").max()
df = df[df["Stratum_Unbekannt"]<200]
startweek = df.index[0].week+53
df.index = range(startweek,startweek+len(df))
f = OrderedDict()
f["Stratum_17_Minus"] = 0
f["Stratum_18_Bis_29"] = 18
f["Stratum_30_Bis_39"] = 30
f["Stratum_40_Bis_49"] = 40
f["Stratum_50_Bis_59"] = 50
f["Stratum_60_Bis_69"] = 60
f["Stratum_70_Bis_79"] = 70
f["Stratum_80_Plus"] = 80
week_icu_max = df.rename(columns=f).filter(f.values())

In [ ]:
week_icu_max.plot()
_ = plt.title("patients in ICU, weekly max.")

In [ ]:
# Change to generate different plots
age = 35

lastweek = meldedatum_hospitalization.index[-1]
print("AgeGroup starting",age,"until week",lastweek)
c = cases.sum("sex").sel(age=age,week=range(1,lastweek+1))
plt.plot(c.coords["week"],c.sel(status="deaths")/c.sel(status="cases"),color="black",label="fraction deaths")
frac_hospital = meldedatum_hospitalization[age]/c.sel(status="cases",week=range(10,92))

plt.plot(range(10,92),frac_hospital,color="tab:blue",label="fraction hospitalized")

plt.fill_between([lastweek-3,lastweek],[0,0],[np.max(frac_hospital)]*2,alpha=.2,color="grey")

plt.xticks([1,14,27,40,54,66,79,92,53*2],labels=[2020,"Q2","Q3","Q4",2021,"Q2","Q3","Q4",2022])
plt.title("CFR and case-hospitalization rate in AG %d by Meldedatum"%age)
plt.ylabel("fraction")
plt.savefig("plots/fraction_AG%d.pdf"%age)
_ = plt.legend()

In [ ]:
plt.semilogy(range(1,lastweek+1),cases.sum("sex").sel(age=age,status="cases",week=range(1,lastweek+1)),color="tab:red",label="cases")
plt.semilogy(range(1,lastweek+1),cases.sum("sex").sel(age=age,status="deaths",week=range(1,lastweek+1)),color="black",label="deaths")
plt.semilogy(meldedatum_hospitalization[age],color="tab:blue",label="hospitalizations")
if age == 80:
    wod = week_of_death[80]+week_of_death[90]
    icu = week_icu_max[80]
elif age == 60:
    wod = week_of_death[60]+week_of_death[70]
    icu = week_icu_max[60]+week_icu_max[70]
elif age == 35:
    wod = week_of_death[30]+week_of_death[40]+week_of_death[50]
    icu = week_icu_max[40]+week_icu_max[50]+week_icu_max[30]
plt.semilogy(range(10,90),wod,"k--",label="deaths (date of death)")
plt.semilogy(range(10,lastweek),reported_deaths.sel(week=range(10,lastweek),age=age).sum("sex"),"-",color="grey",label="deaths (reporting)")
plt.semilogy(icu.index,icu,"purple",label="in ICU (weekly max)")

plt.xticks([1,14,27,40,54,66,79,92,53*2],labels=[2020,"Q2","Q3","Q4",2021,"Q2","Q3","Q4",2022])
plt.fill_between([wod.index[-2],wod.index[-1]+2],[1,1],[np.max(cases.sum("sex").sel(age=age))]*2,alpha=.2,color="grey")
for x in [14,79,69,44]:
    plt.plot([x,x],[1,20000],"-",alpha=.5,color="green")
plt.legend()
plt.ylabel("cases in status")
plt.title("weekly number of cases by Meldedatum in AG %d"%age)
plt.savefig("plots/casesAndStatus_AG%d.pdf"%age)